In [ ]:
# @title
# instalando bobliotecas
!pip install -q selenium
!pip install  -q webdriver-manager
!pip install -q pandas
!pip install -q plyer
!pip install -q lxml html5lib
!pip install -q beautifulsoup4
# Importante bibliotecas
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup as bs
import pandas as pd
from time import sleep
from datetime import datetime
import os
from google.colab import auth
import gspread
from google.auth import default
import requests
from IPython.display import display, HTML

#gerando autentificações do google
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

url = 'https://script.google.com/macros/s/AKfycbyi5Vrhnp4B5tWCvIcsvP7uO3v09jkxNWEMYGric1cGD4I_36PGIUCeIq4DHfden_Za/exec'

#função de ativação de notificação
def notify_colab(title, message):
    """Displays a notification-like message within a Colab notebook."""
    html1 = '''
    <script>
    // Função para falar com voz masculina
    // Configura a voz masculina como padrão
    function falarMensagem(mensagem) {
        const fala = new SpeechSynthesisUtterance(mensagem);
        const vozesDisponiveis = window.speechSynthesis.getVoices();
        const vozMasculinaPadrao = vozesDisponiveis.find(voz =>
            voz.lang.startsWith('pt') && voz.name.toLowerCase().includes('male') // Português e masculina
        );

        if (vozMasculinaPadrao) {
            fala.voice = vozMasculinaPadrao;
        } else {
            console.warn('Nenhuma voz masculina foi encontrada. Usando a voz padrão do sistema.');
        }

        fala.volume = 0.6; // Máximo
        fala.rate = 1.2;   // Velocidade normal
        fala.pitch = 0.3;  // Tom normal
      window.speechSynthesis.speak(fala);
    }
    window.speechSynthesis.onvoiceschanged = () => {

    '''
    html2 = f'''
        falarMensagem("{message}");
        alert("ONU Ativa");
    '''
    html3 = '''
   };


    '''
    abrir_nova_janela= '''
            function abrir(link){
        window.open(link)
    }
        abrir('https://docs.google.com/spreadsheets/d/1lSsYi46ySVXqzmoNJrJIS3rRwkwxB9LRflGeWlokEGE/edit?gid=0#gid=0','_black')

        setTimeout(
            ()=>{
                abrir('https://estoque.brisanet.net.br/#/estoque/transacoes/transacao/liste','_black')

            },1000
        )
    </script>
    '''
    display(HTML(html1 + html2 + html3 + abrir_nova_janela))



#Buscando dados da base de produção do formulário
def buscar_dados():
    lista_n_serie = []
    base_buscar = gc.open('Produção').worksheet('TESTE')
    rows = base_buscar.get_all_values()
    frame = pd.DataFrame.from_records(rows)
    frame = frame.rename(columns=frame.iloc[0])
    frame = frame.drop(frame.index[0])
    frame = frame.dropna()
    onus = frame[frame['Nª SÉRIE'].str.contains('FHTT')]
    dados = onus['Nª SÉRIE'].values.tolist()
    return dados

buscar_dados()

# gerando nova base de dados
def gerar_dados(n_serie):
    base = gc.open('Analise de materiais Ativos').worksheet('Equipamentos_duplicados')
    rows = base.get_all_values()
    frame = pd.DataFrame.from_records(rows)
    frame = frame.rename(columns=frame.iloc[0])
    frame = frame.drop(frame.index[0])
    frame= frame.dropna(how='all')
    diferente_de_nulo = lambda x: x != ''
    frame = list(filter(diferente_de_nulo,frame['Nª SÉRIE'].tolist()))
    base.update_cell(len(frame) +2, 1, n_serie)
    notify_colab('Notificação', 'Uma ó-eni-u ativa')


def imprimindo_resultados(dado, posicao):
    print(f'Dados:{dado} \n Posição:{posicao}') #imprimindo valor vistoriado

# verificando de por onde o robô pode começar
def onde_parei(dados):
        sleep(3)
        visto = gc.open('Analise de materiais Ativos').worksheet('Equipamentos_vistoriados')
        visto_rows = visto.get_all_values()
        visto_frame = pd.DataFrame.from_records(visto_rows)
        frame = visto_frame.rename(columns=visto_frame.iloc[0])
        visto_frame = frame.drop(visto_frame.index[0])
        visto_frame =  visto_frame.values.tolist()
        visto_frame =  visto_frame[-1][0]

        if dados[0] == visto_frame:
            return dados[1:],visto,visto_rows
        elif visto_frame in dados:
            novos_dados = dados[dados.index(visto_frame)+1:]
            return novos_dados,visto,visto_rows
        else:
            return dados,visto,visto_rows
def verificar_se_foi_alterado(n_serie):
    retirados = gc.open('Analise de materiais Ativos').worksheet('Equipamentos_alterados')
    retirados_rows = retirados.get_all_values()
    retirados_frame = pd.DataFrame.from_records(retirados_rows)
    retirados_frame = retirados_frame.rename(columns=retirados_frame.iloc[0])
    retirados_frame = retirados_frame.drop(retirados_frame.index[0])
    retirados_frame = retirados_frame.dropna(how='all')
    if n_serie in retirados_frame['Nº de série'].values.tolist():
        return 'existe'
    else:
        return 'não existe'

# fazendo a verificação se está ativo ou não
def verificando_ativo(driver):
    contagem = 0
    resultado_onde_parei  = onde_parei(buscar_dados())
    for i,dado in enumerate(resultado_onde_parei[0]):
        filtro_n_serie = driver.find_element(By.XPATH,'//*[@id="filters"]')
        sleep(1)
        filtro_n_serie.click()
        sleep(0.5)
        filtro_n_serie.clear()
        sleep(2)
        print(filtro_n_serie.get_attribute('placeholder'))
        filtro_n_serie.send_keys(dado)

        sleep(0.5)
        filtro_n_serie.send_keys(Keys.ENTER)

        sleep(4)
        if verificar_se_foi_alterado(dado) == 'existe':
            pass
        else:
            try:
                pagina_ativo = driver.find_element(By.XPATH,'//*[@id="main"]/div/div/div[2]/div/div[1]/bn-table/div/div[2]/table/tbody')
                verificacao = pagina_ativo.text.split()
                print(verificacao)
                gerar_dados(dado)
                requests.post(url,json={'data': 'ativo'})
            except:
                pass

            resultado_onde_parei[1].update_cell(len(resultado_onde_parei[2]) + contagem, 1, dado)
            contagem += 1
        imprimindo_resultados(dado,[i])




caminho = os.getcwd()
local = {
        "profiledir": rf'{caminho}/.config/Profiles/Luann'
    }
options = webdriver.ChromeOptions()
options.add_argument("--headless")  # Modo invisível
options.add_argument("--no-sandbox")  # Necessário para rodar no Colab
options.add_argument("--disable-dev-shm-usage")  # Necessário para rodar no Colab
options.add_argument("--disable-gpu")

chrome_driver_path = f'{caminho}/chromedriver'
driver = webdriver.Chrome(options=options)
driver.get(f'https://novorevan.brisanet.net.br/#/relatorios/logistica/dispositivos_onus')
sleep(2)




try:
    while True:

        if driver.current_url == 'https://novorevan.brisanet.net.br/#/login':
                driver.refresh()
                sleep(2)
                inserir_cpf = input('Inserir CPF: ')
                inserir_senha = input('Inserir senha: ')
                codigo1 = input('Inserir primeiro código: ')
                codigo2 = input('Inserir segundo código: ')

                documento = driver.find_element(By.XPATH,'//*[@id="usuario"]/input')
                sleep(0.5)
                documento.click()
                sleep(0.5)
                documento.send_keys(inserir_cpf)

                sleep(0.5)

                senha = driver.find_element(By.XPATH,'//*[@id="senha-input"]')
                sleep(0.5)
                senha.click()
                sleep(0.5)
                senha.send_keys(inserir_senha)
                sleep(0.5)

                codigo = driver.find_element(By.XPATH,'//*[@id="slideBox"]/div/div[1]/div[2]/div/div[1]/div[4]/input')
                sleep(0.5)
                codigo.click()
                sleep(0.5)
                codigo.send_keys(codigo1)
                sleep(0.5)
                codigo.send_keys(codigo2)
                sleep(1)
                codigo.send_keys(Keys.ENTER)

                sleep(5)
                if driver.title =='Revan - Relatorios de Dispositivos ONU':
                    print('logado')
                    driver.get(f'https://novorevan.brisanet.net.br/#/relatorios/logistica/dispositivos_onus')
                    driver.refresh()
                    sleep(2)
                    verificando_ativo(driver)
                else:
                    print("não logado")
                sleep(2)



        else:

            verificando_ativo(driver)

except:
    requests.post(url,json={'data': 'parou'})


